In [1]:
# importing the tensorflow package
import tensorflow as tf

In [2]:
# в первую очередь подключим numpy и библиотеку copy, которая понадобится, чтобы сделать deepcopy ряда элементов
import numpy as np
np.random.seed(0)

# IMDB reviews (keras)

In [41]:
from __future__ import print_function

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Dropout
from tensorflow.keras.layers import LSTM, Conv1D, MaxPooling1D
from tensorflow.keras.datasets import imdb

In [4]:
max_features = 20000

# обрезание текстов после данного количества слов (среди top max_features наиболее используемые слова)
maxlen = 80
batch_size = 50 # увеличьте значение для ускорения обучения

In [5]:
print('Загрузка данных...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

Загрузка данных...


In [6]:
# Retrieve the word index file mapping words to indices
word_index = imdb.get_word_index()
# Reverse the word index to obtain a dict mapping indices to words
inverted_word_index = dict((i, word) for (word, i) in word_index.items())
# Decode the first sequence in the dataset
decoded_sequence = " ".join(inverted_word_index[i] for i in x_train[0])

In [7]:
decoded_sequence

"the as you with out themselves powerful lets loves their becomes reaching had journalist of lot from anyone to have after out atmosphere never more room and it so heart shows to years of every never going and help moments or of every chest visual movie except her was several of enough more with is now current film as you of mine potentially unfortunately of you than him that with out themselves her get for was camp of you movie sometimes movie that with scary but pratfalls to story wonderful that in seeing in character to of 70s musicians with heart had shadows they of here that with her serious to have does when from why what have critics they is you that isn't one will very to as itself with other tricky in of seen over landed for anyone of and br show's to whether from than out themselves history he name half some br of 'n odd was two most of mean for 1 any an boat she he should is thought frog but of script you not while history he heart to real at barrel but when from one bit the

In [8]:
print(len(x_train), 'тренировочные последовательности')
print(len(x_test), 'тестовые последовательности')

25000 тренировочные последовательности
25000 тестовые последовательности


In [9]:
len(x_train[0])

218

In [10]:
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [11]:
max_features

20000

In [12]:
layer = Embedding(max_features, 128)

In [13]:
layer(x_train[0]).shape

TensorShape([80, 128])

In [14]:
print('Построение модели...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

Построение модели...


In [15]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [16]:
print('Процесс обучения...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=1, # увеличьте при необходимости
          validation_data=(x_test, y_test))

Процесс обучения...
500/500 [==============================] - 55s 105ms/step - loss: 0.4329 - accuracy: 0.7921 - val_loss: 0.3640 - val_accuracy: 0.8432


In [17]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)

500/500 [==============================] - 7s 13ms/step - loss: 0.3640 - accuracy: 0.8432


In [18]:
print('Результат при тестировании:', score)
print('Тестовая точность:', acc)

Результат при тестировании: 0.36397191882133484
Тестовая точность: 0.8431599736213684


In [19]:
max_features = 20000

# обрезание текстов после данного количества слов (среди top max_features наиболее используемые слова)
maxlen = 200

batch_size = 50 # увеличьте значение для ускорения обучения

In [20]:
print('Загрузка данных...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

Загрузка данных...


In [21]:
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 200)
x_test shape: (25000, 200)


In [22]:
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(tf.keras.layers.Dropout(rate=0.2))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [23]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [24]:
model.fit(x_train, y_train,
          batch_size=50,
          epochs=1, # увеличьте при необходимости
          validation_data=(x_test, y_test))

500/500 [==============================] - 47s 90ms/step - loss: 0.3939 - accuracy: 0.8154 - val_loss: 0.3218 - val_accuracy: 0.8682


In [25]:
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)

500/500 [==============================] - 7s 15ms/step - loss: 0.3218 - accuracy: 0.8682


In [26]:
print('Результат при тестировании:', score)
print('Тестовая точность:', acc)

Результат при тестировании: 0.3218313455581665
Тестовая точность: 0.8682399988174438


# Генерация текста на основе книжки «Алиса в стране чудес»

In [43]:
import numpy as np
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU
from tensorflow.keras.models import Sequential

In [44]:
# построчное чтение из примера с текстом 
with open("alice_in_wonderland.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("ascii", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
chars = set(text)
nb_chars = len(chars)

In [45]:
# создание индекса символов и reverse mapping чтобы передвигаться между значениями numerical
# ID and a specific character. The numerical ID will correspond to a column
# ID и определенный символ. Numerical ID будет соответсвовать колонке
# число при использовании one-hot кодировки для представление входов символов
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

In [46]:
# для удобства выберете фиксированную длину последовательность 10 символов 
SEQLEN, STEP = 10, 1
input_chars, label_chars = [], []

# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])

In [47]:
# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

C:\Users\Анна\AppData\Local\Temp\ipykernel_3736\1321165275.py:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
C:\Users\Анна\AppData\Local\Temp\ipykernel_3736\1321165275.py:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)


In [48]:
# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 128, 128
NUM_ITERATIONS = 10 # 25 должно быть достаточно
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

In [33]:
model = Sequential()
model.add(
    GRU(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

In [34]:
# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель 
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred

Итерация #: 0
1241/1241 [==============================] - 9s 6ms/step - loss: 2.3175
Генерация из посева: very momen
very moment of the was in the was in the was in the was in the was in the was in the was in the was in the was==================================================
Итерация #: 1
1241/1241 [==============================] - 7s 6ms/step - loss: 1.9183
Генерация из посева: toop? soup
toop? soup the dont of the dont of the dont of the dont of the dont of the dont of the dont of the dont of the==================================================
Итерация #: 2
1241/1241 [==============================] - 7s 5ms/step - loss: 1.7562
Генерация из посева: oviding it
oviding it was the grown the reat the reat the reat the reat the reat the reat the reat the reat the reat the ==================================================
Итерация #: 3
1241/1241 [==============================] - 6s 5ms/step - loss: 1.6495
Генерация из посева: walked off
walked off the mormons and the mormons and th

In [76]:
# установка ряда метапамертров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 256, 256
NUM_ITERATIONS = 10 # 25 должно быть достаточно
NUM_EPOCHS_PER_ITERATION = 10
NUM_PREDS_PER_EPOCH = 100

In [77]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [78]:
# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель 
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred

Итерация #: 0
Epoch 1/10
621/621 [==============================] - 24s 37ms/step - loss: 2.5317
Epoch 2/10
621/621 [==============================] - 23s 36ms/step - loss: 2.0783
Epoch 3/10
621/621 [==============================] - 22s 36ms/step - loss: 1.8955
Epoch 4/10
621/621 [==============================] - 22s 36ms/step - loss: 1.7564
Epoch 5/10
621/621 [==============================] - 22s 36ms/step - loss: 1.6523
Epoch 6/10
621/621 [==============================] - 22s 35ms/step - loss: 1.5642
Epoch 7/10
621/621 [==============================] - 23s 37ms/step - loss: 1.4945
Epoch 8/10
621/621 [==============================] - 23s 37ms/step - loss: 1.4342
Epoch 9/10
621/621 [==============================] - 25s 41ms/step - loss: 1.3831
Epoch 10/10
621/621 [==============================] - 24s 39ms/step - loss: 1.3367
Генерация из посева: and then, 
and then, and the project gutenberg-tm electronic works in a long to the project gutenberg-tm electronic works=============